In [1]:
import pandas as pd

train_data_file = "./zhengqi_train.txt"
test_data_file =  "./zhengqi_test.txt"

train_data = pd.read_csv(train_data_file, sep='\t', encoding='utf-8')
test_data = pd.read_csv(test_data_file, sep='\t', encoding='utf-8')

In [2]:
epsilon = 1e-5
func_dict = {'add':lambda x,y:x+y,'mins':lambda x,y:x-y,'div':lambda x,y:x/(y+epsilon),'multi':lambda x,y:x*y}

In [3]:
def auto_features_make(train_data, test_data, func_dict, col_list):
    train_data, test_data = train_data.copy(),test_data.copy()
    for col_i in col_list:
        for col_j in col_list:
            for func_name, func in func_dict.items():
                for data in [train_data, test_data]:
                    func_features = func(data[col_i], data[col_j])
                    col_func_features = '-'.join([col_i, func_name, col_j])
                    data[col_func_features]= func_features
    return train_data, test_data

In [4]:
train_data2, test_data2 = auto_features_make(train_data, test_data, func_dict, col_list=test_data.columns)

In [5]:
from sklearn.decomposition import PCA
pca = PCA(n_components=500)
train_data2_pca = pca.fit_transform(train_data2.iloc[:,0:-1])
test_data2_pca = pca.fit_transform(test_data2)
train_data2_pca = pd.DataFrame(train_data2_pca)
train_data2_pca['target'] = train_data2['target']

In [6]:
X_train2 = train_data2[train_data2.columns].values
y_train = train_data2['target']

In [7]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import numpy as np

Folds = 5
kf = KFold(len(train_data2), n_splits=Folds, random_state=231, shuffle=True)
MSE_DICT = {'train_mse':[], 'test_mse':[]}

for i, (train_index, test_index) in enumerate(kf.split(X_train2)):
    lgb_reg = lgb.LGBMRegressor(learning_rate = 0.01, max_depth=1, n_estimators=5000, boosting_type='gbdt', random_state=2131, objective='regression')
    X_train_Kfold, X_test_Kfold = X_train2[train_index], X_train2[test_index]
    y_train_Kfold, y_test_Kfold = y_train[train_index], y_train[test_index]

    lgb_reg.fit(X=X_train_Kfold, y=y_test_Kfold, eval_set=[(X_train_Kfold, y_train_Kfold),(X_test_Kfold, y_test_Kfold)], eval_names=['Train','Test'], early_stopping_rounds=100, eval_metric='MSE', verbose=50)
     # 训练集预测 测试集预测
    y_train_KFold_predict = lgb_reg.predict(X_train_KFold,num_iteration=lgb_reg.best_iteration_)
    y_test_KFold_predict = lgb_reg.predict(X_test_KFold,num_iteration=lgb_reg.best_iteration_) 
    
    print('第{}折 训练和预测 训练MSE 预测MSE'.format(i))
    train_mse = mean_squared_error(y_train_KFold_predict, y_train_KFold)
    print('------\n', '训练MSE\n', train_mse, '\n------')
    test_mse = mean_squared_error(y_test_KFold_predict, y_test_KFold)
    print('------\n', '预测MSE\n', test_mse, '\n------\n')
    
    MSE_DICT['train_mse'].append(train_mse)
    MSE_DICT['test_mse'].append(test_mse)
print('------\n', '训练MSE\n', MSE_DICT['train_mse'], '\n', np.mean(MSE_DICT['train_mse']), '\n------')
print('------\n', '预测MSE\n', MSE_DICT['test_mse'], '\n', np.mean(MSE_DICT['test_mse']), '\n------')

OSError: dlopen(/Users/sparklingdeng/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/sparklingdeng/opt/anaconda3/lib/python3.8/site-packages/lightgbm/lib_lightgbm.so
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)